# Tuning Gemma 7b with ORPO and QLora on your laptop to enhance Chinese capability

## Overview

This tutorial can run on your laptop with NVIDIA GPU. 
you should install CUDA 12.3，Pycharm/VSode and PyTorch 2.2.1 beforehand.

Dataset: wenbopan/Chinese-dpo-pairs

## Setup
### download gemma-7b model from huggingface
[https://huggingface.co/google/gemma-7b/tree/main](https://huggingface.co/google/gemma-7b/tree/main)
Note: I don't like the cache model mechanism of huggingface, 

### Configure your wandb key

To use wandb to monitor, you must provide wandb API key. you can apply API key from [https://wandb.ai](https://wandb.ai)

### Configure your Hugging Face access token

if you want to upload your tuned LLM to Hugging Face, you can apply access token (write) from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

### Set environment variables

Set environment variables for `wandb` and `huggingface`

In [1]:
#make sure that CUDA works
import torch

torch.cuda.is_available(), torch.version.cuda

(True, '12.1')

In [1]:
# load wenbopan/Chinese-dpo-pairsdataset

from datasets import load_dataset


#dataset = load_dataset("argilla/distilabel-capybara-dpo-7k-binarized",split="train")
#dataset2 = load_dataset("allenai/ultrafeedback_binarized_cleaned",split="train")

dataset = load_dataset("wenbopan/Chinese-dpo-pairs",split="train")


dataset[0]

#dataset.to_csv("a.csv")

{'prompt': '任务定义：您将获得一个亚马逊食品产品的评论以及其极性（积极或消极）。您的任务是回答“True”，如果指定的句子及其极性匹配；否则，回答“False”。\n问题：我买了这些很长时间，因为我认为它们是美国原产的，所以对这个产品感觉更好。我购买了几袋（条状、包装的苹果等），现在意识到原产地是中国。考虑到所有关于包装的FDA法规，是时候要求原产地在包装的正面显著显示（比如20号字体 - 而不是微不足道的写法，没有人会注意到！）！因此，我将不再购买Dogswell产品。另外，生皮更糟糕，因为狗可以摄入更高剂量的任何用于处理皮革的毒素（而且你可以肯定有很多！）。我甚至拿起一包名为（类似）“U.S.A.生皮”的产品，背面微小的字体写着“中国制造”。为什么我们的立法者不采取一些有益的行动，打击这种行为？这最多是狡猾，最坏的情况下是故意欺骗！亚马逊，请为我们的“毛孩子”找到不会让它们生病或致命的产品！\n极性：积极\n\n解决方案：False\n\n问题：您想从花生酱中去除糖而不是脂肪。老实说，与之相比，这个味道很糟糕。花生中大约50％的脂肪是单不饱和脂肪。不饱和脂肪可以帮助降低血液中的LDL胆固醇（“坏”胆固醇）水平，而不影响HDL胆固醇（“好”胆固醇）。研究表明，食用更高比例的单不饱和脂肪的人降低了患心脏病、哮喘、阿尔茨海默病、乳腺癌甚至抑郁症的风险。只需购买天然种类，避免添加剂。\n极性：积极\n\n解决方案：False\n\n问题：我在意识到我得到了多少之前就买了这个，也在知道杂货店有这个产品之前！那是我的错。这些种子很棒，只是我冰箱里有太多了！\n极性：消极\n\n解决方案：',
 'system': '',
 'chosen': '错误\n\n问题：我订购了这些牛肉棒作为肉干的健康替代品，但很失望地发现它们含有添加糖。我尽量避免摄入添加糖，所以我不能吃这些。在亚马逊上的配料中没有列出这一点，所以我建议在购买前检查包装。\n\n极性：负面\n\n解决方案：',
 'rejected': '真的',
 'source': 'flan_v2_niv2',
 'id': None}

In [2]:
#format dataset format

from datasets import load_dataset
from transformers import AutoTokenizer

def chatml_format(example):
    message = {"role": "user", "content": example['system']+example['prompt']}
    # Format instruction
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
    # Format chosen answer
    chosen = example['chosen']+tokenizer.eos_token
    # Format rejected answer
    rejected = example['rejected']+tokenizer.eos_token

    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# Load dataset
dataset = load_dataset("wenbopan/Chinese-dpo-pairs",split="train")
#dataset = load_dataset("argilla/distilabel-intel-orca-dpo-pairs", split="train")
#dataset = dataset.filter(
#   lambda r: 
#       r["status"] != "tie" and 
#       r["chosen_score"] >= 5
#       and not r["in_gsm8k_train"]
#)
# Save columns
original_columns = dataset.column_names


# Tokenizer
model_name = "c:/ai/models/gemma"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


# Format dataset
dataset = dataset.map(
    chatml_format,
    remove_columns=original_columns
)

# Print sample
dataset[1]

{'prompt': '<bos><start_of_turn>user\n任务定义：给出一段泰米尔语的文本。将其从泰米尔语翻译成英语。翻译不能省略或添加原句的信息。\n问题：கேரளாவின் வெள்ள நிலைமைகளை ஆய்வு செய்ய இரண்டு நாட்களில் இரண்டாவது முறையாக தேசிய நெருக்கடி மேலாண்மைக் குழு கூடியது\n\n解决方案：国家内政部 NCMC 在两天内第二次会议上，审查喀拉拉邦的洪水情况\n\n问题：வெகு விரைவில் உங்களது வீட்டின் சாவிகள் உங்களுக்கு கிடைக்கும் என்று நான் உறுதி கூறுகிறேன்” என்றும் பிரதமர் தெரிவித்தார்.\n\n解决方案：我向您保证，很快您将拥有您家的钥匙。\n\n问题：இஸ்ரோவின் தொழில்நுட்ப தொடர்பு கூடத்தில் பணிபுரியவுள்ள தொடர்பு அலுவலர், ரஷ்யாவிலும், அண்டை நாடுகளிலும், இஸ்ரோவின் செயல்திட்ட இலக்குகள் தொடர்பான பணிகளை ஒருங்கிணைப்பார்.\n\n解决方案：以色列的技术合作伙伴，包括俄罗斯、印度等国家，将与以色列的行动计划相关的活动进行协调。<end_of_turn>\n<start_of_turn>model\n',
 'chosen': '当然，我很乐意帮助你！给定的泰米尔语句子“கேரளாவின் வெள்ள நிலைமைகளை ஆய்வு செய்ய இரண்டு நாட்களில் இரண்டாவது முறையாக தேசிய நெருக்கடி மேலாண்மைக் குழு கூடியது” 可以翻译成英文为“The Ministry of Home Affairs NCMC meets for the second time in two days to review the flood situation in Kerala.”\n句子“வெகு விரைவில் உங்களது வீட்டின் சாवிකள் உங்களுக்கு கிடைக்கும் என்று

In [ ]:
dataset

In [3]:
#Using ORPOTrainer and QLora to tune Gemma 7B to enhance Chinese capability
import os
import gc
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
import wandb
from dotenv import load_dotenv, find_dotenv
from trl import ORPOTrainer
from trl import ORPOConfig

#init env
env =load_dotenv(find_dotenv())
hf_token = os.getenv("huggingface")
wb_token = os.getenv('wandb')
wandb.login(key=wb_token)

#local model path
local_model_path ="c:/ai/models/gemma"
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage =True,
    #torch_dtype="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
    quantization_config= BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
    )
)
model.config.use_cache = False

new_model = "lion-gemma-7b-cn"

torch.cuda.empty_cache()


# Training arguments
training_args = ORPOConfig (
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':True},
    remove_unused_columns=False,
    learning_rate=8e-5,
    lr_scheduler_type="cosine",
    #max_steps=400,
    num_train_epochs=1,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="adamw_bnb_8bit",
    warmup_steps=40,
    bf16=True,
    max_prompt_length=256,
    max_length=1024,
    report_to="wandb",
)

# Create DPO trainer
orpo_trainer = ORPOTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config
)

# Fine-tune model with ORPO
orpo_trainer.train()

wandb: Currently logged in as: kevon-zeng. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kevon\.netrc
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss


TrainOutput(global_step=670, training_loss=2.0626583168755714, metrics={'train_runtime': 15616.5652, 'train_samples_per_second': 0.687, 'train_steps_per_second': 0.043, 'total_flos': 0.0, 'train_loss': 2.0626583168755714, 'epoch': 1.0})

In [4]:
# save tuning checkpoint

final_checkpoint = "gemma_final_checkpoint"
orpo_trainer.model.save_pretrained(final_checkpoint)
tokenizer.save_pretrained(final_checkpoint)


C:\dev\anaconda3\envs\FineTuning_CUDA12\Lib\site-packages\peft\utils\save_and_load.py:148: UserWarning: Could not find a config file in c:/ai/models/gemma - will assume that the vocabulary was not modified.
  warnings.warn(


('gemma_final_checkpoint\\tokenizer_config.json',
 'gemma_final_checkpoint\\special_tokens_map.json',
 'gemma_final_checkpoint\\tokenizer.model',
 'gemma_final_checkpoint\\added_tokens.json',
 'gemma_final_checkpoint\\tokenizer.json')

In [5]:
# merge checkpoint with original llm
env =load_dotenv(find_dotenv(),override=True)
hf_token = os.getenv("huggingface")

#Flush memory
del orpo_trainer, model
gc.collect()
torch.cuda.empty_cache()

# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    return_dict=True,
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, final_checkpoint)
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

('lion-gemma-7b-cn\\tokenizer_config.json',
 'lion-gemma-7b-cn\\special_tokens_map.json',
 'lion-gemma-7b-cn\\tokenizer.model',
 'lion-gemma-7b-cn\\added_tokens.json',
 'lion-gemma-7b-cn\\tokenizer.json')

In [6]:
# Push them to the HF Hub
os.environ["HTTPS_PROXY"] ="http://127.0.0.1:7890"
model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

...

C:\dev\anaconda3\envs\FineTuning_CUDA12\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kevon\.cache\huggingface\hub\models--KeyonZeng--lion-gemma-7b-cn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KeyonZeng/lion-gemma-7b-cn/commit/ed194b039d098d4e0eaaddcf2d99bdaad52de5bc', commit_message='Upload tokenizer', commit_description='', oid='ed194b039d098d4e0eaaddcf2d99bdaad52de5bc', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
#test tuned gemma-7b model

from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import torch

new_model = "lion-gemma-7b-cn"
tokenizer = AutoTokenizer.from_pretrained(new_model,torch_dtype=torch.bfloat16, device_map="cuda")
model = AutoModelForCausalLM.from_pretrained(
    new_model,
    torch_dtype=torch.bfloat16,
    device_map="cuda", 
    quantization_config= BitsAndBytesConfig
        (
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
    ))  # You may want to use bfloat16 and/or move to GPU here

messages = [
    {"role": "user", "content": "请向小学生解释一下什么是大语言模型"},
 ]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")

tokenized_chat =tokenized_chat.to('cuda')
outputs = model.generate(tokenized_chat, max_new_tokens=128, temperature = 0.1)
print(tokenizer.decode(outputs[0]))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<bos><start_of_turn>user
请向小学生解释一下什么是大语言模型<end_of_turn>
<start_of_turn>model
大语言模型是一种特殊的计算机程序，可以像我们一样说话，理解和生成文本。它们可以学习很多东西，比如我们人类可以学习的知识，比如语言、数学和科学。

大语言模型可以帮助我们做很多事情，比如回答问题、提供信息、帮助我们学习新的东西，甚至创作故事。它们可以帮助我们理解世界，并帮助我们做出更好的决定。

大语言模型可以学习很多东西，比如我们人类可以学习的知识，比如语言、数学和科学。它们可以帮助我们做很多事情，比如回答问题、提供信息、帮助我们学习新的东西，甚至


In [17]:
# compare generated content by gemma-7b base model
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import torch

old_model = "c:/ai/models/gemma"
tokenizer = AutoTokenizer.from_pretrained(new_model,torch_dtype=torch.bfloat16, device_map="cuda")
model = AutoModelForCausalLM.from_pretrained(
    old_model,
    torch_dtype=torch.bfloat16,
    device_map="cuda", 
    quantization_config= BitsAndBytesConfig
        (
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
    ))  # You may want to use bfloat16 and/or move to GPU here

messages = [
    {"role": "user", "content": "请向小学生解释一下什么是大语言模型"},
 ]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")

tokenized_chat =tokenized_chat.to('cuda')
outputs = model.generate(tokenized_chat, max_new_tokens=128, temperature = 0.1 )
print(tokenizer.decode(outputs[0]))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<bos><start_of_turn>user
请向小学生解释一下什么是大语言模型<end_of_turn>
<start_of_turn>model
大语言模型 (LLM) 是一个特殊的计算机程序，可以像一个人一样理解和生成大量的文本信息。

**大语言模型的主要特点:**

* **学习大量的文本数据:** LLM 在训练过程中，会学习大量的文本数据，包括书籍、文章、代码和网页上的内容。
* **生成文本:** LLM 可以根据输入信息生成新的文本，例如故事、文章、代码或翻译。
* **理解语言:** LLM 可以理解语言结构和语法，以及文本的意思。
* **对话:** LLM 可以与人进行对话，并提供信息、帮助或娱乐。
